In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from tqdm import tqdm
import sys
sys.path.append('/fs/cbsuvlaminck2/workdir/bmg224/hiprfish/image_analysis_code')
import image_functions as imfn
# from segmentation import Segmentation


In [4]:
# from image import Image

In [3]:
cluster_path = '/fs/cbsuvlaminck2/'

# data_dir = cluster_path + '/workdir/bmg224/data/2021/devlaminck_widefield/2021_04_05_dnafishmethodchar'
out_dir = '../image_processing'
sample_names = imfn.get_sample_names(data_dir=out_dir, sample_glob='*Stitch', ext='.npy')
refl = ['gfp', 'cy5']
factors = ['exp','method', 'plasmid','fov']
flat_field_filename = ''
# flat_field_filename = '/fs/cbsuvlaminck2/workdir/bmg224/data/2021/devlaminck_widefield/2021_04_12_flat_field_correction_lens_100x_refl_cy5.czi'
print(len(sample_names))
print(sample_names[0])

47
2021_06_02_plasmidairyscan_exp_040421_method_f_plasmid_neg_fov_tile1_Airyscan_Processing_Stitch


In [4]:
keys =[imfn.get_filename_keys(sn, factors) for sn in sample_names]
len(keys)

47

In [5]:
keys[0]

['040421', 'f', 'neg', 'tile1']

In [10]:
import numpy as np
import os

In [ ]:
# Get segs and raws
spot_segs, spot_raws, cell_segs, cell_raws = [],[],[],[]
for sn in tqdm(sample_names):
    ss, sr, cs, cr = [],[],[],[]
    spot_split_folder = out_dir + '/' + sn + '_spot_seg_split'
    cell_split_folder = out_dir + '/' + sn + '_cell_seg_split'
    spot_split_fns = imfn.get_sample_names(data_dir=spot_split_folder, sample_glob='*col_[012]', ext='.npy')
    cell_split_fns = imfn.get_sample_names(data_dir=cell_split_folder, sample_glob='*col_[012]', ext='.npy')
    spot_split_fns = [spot_split_folder + '/' + i for i in spot_split_fns]
    cell_split_fns = [cell_split_folder + '/' + i for i in cell_split_fns]
    # for each split file
    for sfn, cfn in zip(spot_split_fns, cell_split_fns):
        ss_fn = sfn + '_seg.npy'
        sr_fn = sfn + '.npy'
        cs_fn = cfn + '_seg.npy'
        cr_fn = cfn + '.npy'
        # Load cell and spot numpy segs and raw
        ss.append(np.load(ss_fn))
        cs.append(np.load(cs_fn))
        sr.append(np.load(sr_fn))
        cr.append(np.load(cr_fn))
    spot_segs.append(ss)
    spot_raws.append(sr)
    cell_segs.append(cs)
    cell_raws.append(cr)
    

 98%|█████████▊| 46/47 [00:18<00:00,  2.99it/s]

In [8]:
# Get cell segmentation
import numpy as np
output_dir = '../image_processing'
cell_seg_filename = [output_dir + '/' + sn + '_cell_seg.npy' for sn in sample_names]
cell_seg = [np.load(csf) for csf in cell_seg_filename]

In [9]:
# Get Raw spot images (flat field corrected)
from image import Image
raw_corrected = []
for sn in sample_names:
    im = Image(sample_name=sn, illumination_names=refl, data_dir=data_dir, 
               flat_field_filename=flat_field_filename)
    im.pick_channel('cy5')
    im.flat_field_correction()
    raw_corrected.append(im.raw_2D)

In [10]:
# Get bg pixels
bg_pixels = [rc[cs == 0] for rc, cs in zip(raw_corrected, cell_seg)]

In [11]:
# Calculate median background pixel intensity
mean_bg_pix_int = [np.median(bp) for bp in bg_pixels]
print(mean_bg_pix_int[0])

0.15383656


In [12]:
# load csvs
import pandas as pd
spot_props_filenames = [output_dir + '/' + sn + '_spot_seg_cell_id.csv' for sn in sample_names]
spot_props = [pd.read_csv(spf) for spf in spot_props_filenames]

In [13]:
# filter spots not in cells
spot_props_cell = [sp[sp.cell_id != 0] for sp in spot_props]

In [14]:
# Norm by bg pixel intensity
spot_props_norm = []
mins, maxs = [],[]
for sc, mbpi in zip(spot_props_cell, mean_bg_pix_int):
# for sc, mbpi in zip(spot_props, mbpi_new):
    scc = sc.copy()
    scc['int_norm'] = scc['Intensity']/mbpi
    maxs.append(scc.int_norm.max())
    mins.append(scc.int_norm.min())
    spot_props_norm.append(scc)

In [15]:
# Generate histograms

hists = []
rnge = (0, np.max(maxs))
for sp in spot_props_norm:
    values = sp.int_norm.values
    y, bin_edges = np.histogram(values, range=rnge, bins=100)
    hists.append(y)

# hists = []
# rnge = (0, 1)
# for sp in spot_props_cell:
#     values = sp.Intensity.values
#     y, bin_edges = np.histogram(values, range=rnge, bins=100)
#     hists.append(y)
    
intensity_values = 0.5*(bin_edges[1:]+bin_edges[:-1])

In [16]:
# Generate dictionary of histograms
hist_dict = imfn.get_nested_dict(keys, hists, [0,1,2])

In [27]:
# Set up ordered lists of factors
I = ['b','c','g']
J = ['pos','neg']
K = ['1','2','3']

In [28]:
# Calculate the mean and stdev between fields of view for each set of factors
# Generate a dictionary of mean and stdev values
plot_dict = {}
for i in I:
    plot_dict[i] = {}
    for j in J:
        for k in K:
            hist_list = [hist_dict[i][j][k][0][1] for k in K]
        hist_stack = np.vstack(hist_list)
        means = np.mean(hist_stack, axis=0)
        stdevs = np.std(hist_stack, axis=0)
        plot_dict[i][j] = (means, stdevs)

In [29]:
# # By FOV
# plot_dict = {}
# for i in I:
#     plot_dict[i] = {}
#     for j in J:
#         plot_dict[i][j] = {}
#         for k in K:
#             plot_dict[i][j][k] = hist_dict[i][j][k][0][1]


In [40]:
# plot parameters
import image_plots as ip
import numpy as np
import matplotlib.pyplot as plt

fig_dir = '../figures'

# col = 'w'
# dims = (10,5)
# lw = 4
# ft = 20
# save=False
# labelbottom=True

col = 'k'
lw = 1
ft = 5
dims = (0.9843 ,0.5906)
save=True
labelbottom=False

# colors = plt.rcParams['axes.prop_cycle'].by_key()['color']


In [42]:
# Generate PLots
for i in I:
    print('method: ', i)
    fig, ax = ip.general_plot(col=col, dims=dims, lw=lw)
    # Plot one line for each control
    line_styles = ['-',':']
    legend_labels = ['Postitive','Negative']
    line_colors = [colors[1],colors[0]]
    for j, ls, ll, lc in zip(J, line_styles, legend_labels, line_colors):
#         for k in K:
#             means = hist_dict[i][j][k][0][1]
        
        means, stdevs = plot_dict[i][j]
        ax.plot(intensity_values, np.log10(means+1), linestyle=ls, lw=lw*0.5, label=ll, c=lc)
        ax.fill_between(intensity_values, np.log10(means-stdevs+1),np.log10(means + stdevs+1), color=lc, alpha=0.5)
#         ax.plot(intensity_values, means, linestyle=ls, lw=lw*0.5, label=ll, c=lc)
#         ax.fill_between(intensity_values, means-stdevs, means + stdevs, color=lc, alpha=0.5)
#         for k in K:
#             means = plot_dict[i][j][k]
#             ax.plot(intensity_values, means, linestyle=ls, lw=lw*0.5, label=ll, c=lc)

    if i == 'a':
        ax.legend(fontsize=ft)
             
#     ax.set_xlim((0.5,3))
    ax.set_ylim(-0.3, 4)
    ax.set_yticks([1,2,3])
    ax.tick_params(labelbottom=labelbottom, bottom=True, direction='in', labelsize=ft, color=col, labelcolor=col, length=lw*2)
#     if i == 'g':
#         ax.tick_params(labelbottom=True, bottom=True, direction='in', labelsize=ft, color=col, labelcolor=col, length=lw*2)
    filename = fig_dir + '/spot_int_histogram_method_' + i + '.pdf'
    if save:
        ip.plt.savefig(filename, transparent=True)
    else:
        ip.plt.show()
    ip.plt.close()


method:  b
method:  c
method:  g


/home/bmg224/miniconda3/envs/hiprfish_imaging/lib/python3.5/site-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in log10
  from ipykernel import kernelapp as app
